In [28]:
import numpy as np
from numpy import dot
from numpy.linalg import eigh, eig
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from scipy.optimize import curve_fit

In [2]:
def lorentz(X, mean, sigma, scale, offset):
    return scale/(np.pi * sigma * (1 + ((X - mean)/sigma)**2)) + offset

def lorentz_diff1(X, mean, sigma, scale, offset):
    return -(2 * sigma * (-mean + X)) / (np.pi * (mean**2 + sigma**2 - 2*mean*X + np.power(X, 2))**2) * scale + offset

def lorentz_diff2(X, mean, sigma, scale, offset):
    return -(2 * sigma * (sigma**2 - 3 * np.power(mean - X, 2))) / (np.pi * np.power(sigma**2 + np.power(mean - X, 2), 3)) * scale + offset

def lorentz_diff3(X, mean, sigma, scale, offset):
    return (-(48 * np.power(-mean + X, 3))/(sigma**7 * np.pi * np.power(1 + np.power(-mean + X, 2)/sigma**2, 4)) + (24 * (-mean + X))/(sigma**5 * np.pi * np.power(1 + np.power(-mean + X, 2)/sigma**2, 3))) * scale + offset

def lorentz_spread(X, mean, sigma, scale, offset):
    return (mean**2 - 2 * mean * X + np.power(X, 2) - sigma**2)/(np.pi * (mean**2 - 2 * mean * X + np.power(X, 2) + sigma**2)**2) * scale + offset

def line(X, offset):
    return np.ones(len(X)) * offset

# def both_lorentz(X, mean1, sigma1, scale1, offset1, mean2, sigma2, scale2, offset2):
#     return lorentz(X, mean1, sigma1, scale1, offset1) + lorentz_shift(X, mean2, sigma2, scale2, offset2)

class Map:
    def __init__(self, src, x, y, **kwargs):
        raw_data = np.loadtxt(src, delimiter='\t')
        if 'begin' in kwargs and 'end' in kwargs:
            raw_data = raw_data[np.r_[0:1, (kwargs['begin']+2):(kwargs['end']+2)]]
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(raw_data[2:], columns = ['RamanShift'] + list(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T

        self.cov_matrix = np.cov(self.data_matrix)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        
        self.base_vectors = np.dot(self.vectors, self.data_matrix)
    
    def reconstruct(self, count):
        return np.dot(self.vectors[:count].T, self.base_vectors[:count])
    
    def plotComponent(self, num):
        fig = px.imshow(np.reshape(self.vectors[num], self.shape), title ='Komponent ' + str(num))
        return fig
    
    def plotPhysicalComponent(self, v):
        img = np.dot(np.array(v), self.vectors[:len(v)])
        fig = px.imshow(np.reshape(img, self.shape), title ='Peak')
        return fig
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self, cum, log):
        if cum:
            fig = px.bar(np.cumsum(self.weights), title = 'Wagi', log_y=log)
        else:
            fig = px.bar(self.weights, title = 'Wagi', log_y=log)
        return fig
    
    def plotSpectrums(self, ids, **kwargs):
        fig = go.Figure()
        if 'base' not in kwargs:
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = self.data_matrix[num], mode='lines', name = 'ID: ' + str(num))
        else:
            r = self.reconstruct(kwargs['base'])
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = r[num], mode='lines', name = 'ID: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
    
    def plotSpectrum(self, num, **kwargs):
        return self.plotSpectrums([num], **kwargs)
    
    def plotBaseVectors(self, count):
        fig = go.Figure()
        for num in range(count):
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[num], mode='lines', name = 'Num: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
            
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: np.linalg.norm([x, y] - e), self.map)))
    
    def posToId(self, x, y):
        return self.shape[0] * y + x
    
    def draw_curve(self, curve, params, a, b, **draw):
        fig = go.Figure()
        if 'spectrum' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.data_matrix[draw['spectrum']], mode='lines')
        if 'base_vector' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[draw['base_vector']], mode='lines')
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        fig.add_scatter(mode='lines', x = data_slice, y = curve(data_slice, *params))
        fig.update_layout(hovermode="x")
        return fig
    
    def fit_curve(self, curve, p0, a, b, seek, **plot):
        y = 0
        if 'spectrum' in plot:
            y = self.data_matrix[plot['spectrum'], (self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]
        if 'base_vector' in plot:
            y = self.base_vectors[plot['base_vector'], (self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        
        bounds = 0
        if 'bounds' in plot and plot['bounds'] == 'inf':
            bounds = ((-np.inf, -np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf, np.inf))
        else:
            bounds = ((p0[0]-2, -np.inf, -np.inf, -np.inf), (p0[0]+2, np.inf, np.inf, np.inf))
        
        param, cov = curve_fit(curve, data_slice, y, p0=p0, bounds = bounds)
        print(param)
        if 'draw' in plot and plot['draw'] == True:
            self.draw_curve(curve, param, a, b, **plot).show()
        return param[seek]

In [3]:
map_b = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_B_trojkat3_mapa_10x10um_step_0.3um_data.txt', 34, 34, begin=250, end=450)

In [ ]:
map_b.plotWeights(False, True).show()

In [ ]:
map_b.plotBaseVectors(7).show()

In [4]:
## Map A
t_matrix = np.zeros((7,7))

draw = False
# for i in range(6):
#     t_matrix[0][i] = np.average(map_a.base_vectors[i], weights=((map_a.raman_shift < 1310) | (map_a.raman_shift > 1420)) )
    # t_matrix[0][i] = map_a.draw_curve(line, [np.average(map_a.base_vectors[i], weights=((map_a.raman_shift < 1310) | (map_a.raman_shift > 1420)) )], 1200, 1500, base_vector = i, draw = True, bounds = 'inf').show()

t_matrix[0][0] = map_b.fit_curve(lorentz, (1392, 2.5, 17 * 10**5, -7000), 1380, 1400, 2, base_vector = 0, draw = draw)

t_matrix[1][0] = map_b.fit_curve(lorentz, (1345, 15,  5 * 10**5, -10000), 1330, 1360, 2, base_vector = 0, draw = draw)
t_matrix[1][3] = map_b.fit_curve(lorentz, (1343, 15,  -5 * 10**5, 1000), 1335, 1352, 2, base_vector = 3, draw = draw)
t_matrix[1][4] = map_b.fit_curve(lorentz, (1345, 15,  -5 * 10**5, 1000), 1335, 1352, 2, base_vector = 4, draw = draw)
t_matrix[1][5] = map_b.fit_curve(lorentz, (1343, 15,  5 * 10**5, 1000), 1335, 1352, 2, base_vector = 5, draw = draw)
t_matrix[1][6] = map_b.fit_curve(lorentz, (1345, 15,  5 * 10**5, 1000), 1335, 1352, 2, base_vector = 6, draw = draw)

t_matrix[2][2] = -3 * 10**5 #map_b.fit_curve(lorentz, (1367, 28, 3 * 10**5, -4500), 1358, 1377, 2, base_vector = 2, draw = True)
t_matrix[2][3] = map_b.fit_curve(lorentz, (1369, 10, 2.4 * 10**5, -1400), 1355, 1380, 2, base_vector = 3, draw = draw)
t_matrix[2][4] = -3 * 10**5 #map_b.fit_curve(lorentz, (1369, 10, -2.4 * 10**5, -1400), 1355, 1380, 2, base_vector = 4, draw = True)
# t_matrix[2][5] = map_b.fit_curve(lorentz, (1369, 10, -2.4 * 10**5, -1400), 1355, 1380, 2, base_vector = 5, draw = True)


t_matrix[3][1] = map_b.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 1, draw = draw, bounds = 'inf')
t_matrix[3][2] = map_b.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 2, draw = draw, bounds = 'inf')

t_matrix[4][3] = map_b.fit_curve(lorentz_diff2, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 3, draw = draw, bounds = 'inf')
t_matrix[4][4] = map_b.fit_curve(lorentz_diff2, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 4, draw = draw, bounds = 'inf')
t_matrix[4][5] = map_b.fit_curve(lorentz_diff2, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 5, draw = draw, bounds = 'inf')

t_matrix[5][6] = map_b.fit_curve(lorentz_diff3, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 6, draw = draw, bounds = 'inf')

t_matrix[6][1] = t_matrix[3][1] * -.1
t_matrix[6][2] = t_matrix[3][2]

t_matrix

[1.39202002e+03 2.62826908e+00 1.11042169e+06 2.81143717e+03]
[1.34440756e+03 1.47239179e+01 6.28275855e+05 6.71794929e+03]
[ 1.34127243e+03  2.82401283e+01 -5.53753090e+05  6.99886713e+03]
[ 1.34700000e+03  1.73882835e+01 -1.71714125e+05  5.50593369e+02]
[ 1.34270426e+03  1.79562924e+01  2.95059275e+05 -4.62596530e+03]
[ 1.34489280e+03  1.55636300e+01  4.93482690e+04 -1.07277503e+03]
[ 1.36941988e+03  2.28789241e+01  3.88613578e+05 -1.93364240e+03]
[ 1.39243402e+03  2.84168003e+00  6.95479778e+05 -3.34496767e+03]
[ 1.39178773e+03  2.37631257e+00 -5.16878925e+04 -6.28325869e+03]
[1.39251948e+03 3.24766392e+00 4.88395682e+05 3.58080503e+03]
[1.39242386e+03 2.68432137e+00 9.00429640e+04 3.25677794e+02]
[ 1.39287671e+03  2.72608687e+00  1.23086478e+05 -1.43647140e+03]
[ 1.39276101e+03  2.77781698e+00  1.14975752e+05 -6.37537521e+02]


array([[1110421.69099246,       0.        ,       0.        ,
              0.        ,       0.        ,       0.        ,
              0.        ],
       [ 628275.85540328,       0.        ,       0.        ,
        -553753.09016809, -171714.12491174,  295059.27483802,
          49348.26899532],
       [      0.        ,       0.        , -300000.        ,
         388613.57843404, -300000.        ,       0.        ,
              0.        ],
       [      0.        ,  695479.77812915,  -51687.89253901,
              0.        ,       0.        ,       0.        ,
              0.        ],
       [      0.        ,       0.        ,       0.        ,
         488395.68172592,   90042.96395163,  123086.47753958,
              0.        ],
       [      0.        ,       0.        ,       0.        ,
              0.        ,       0.        ,       0.        ,
         114975.75225759],
       [      0.        ,  -69547.97781291,  -51687.89253901,
              0.        ,       

In [ ]:
map_b.draw_curve(lorentz, (1367, 28, -3 * 10**5, -4500), 1358, 1377, base_vector = 2).show()


In [57]:
rm1 = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, -.8, .4, 0, 1, 0, 0],
    [0, -.2, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

rm2 = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 2, 4, 0, -1, .1, 1],
])

rm3 = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, -.1],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

rm4 = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [5, 0, 1, 0, 0.2, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
])


tmp = np.dot(np.linalg.inv(t_matrix), dot(rm4.T, dot(rm3.T, dot(rm2.T, rm1.T))))

new_base_vectors = np.dot(map_b.base_vectors[:7].T, tmp).T
new_vectors = np.dot(tmp, map_b.vectors[:7])

px.line(x = map_b.raman_shift, y = new_base_vectors[0]).show()
# px.imshow(np.reshape(new_vectors[0], map_a.shape)).show()
px.line(x = map_b.raman_shift, y = new_base_vectors[1]).show()
# px.imshow(np.reshape(new_vectors[1], map_a.shape)).show()
px.line(x = map_b.raman_shift, y = new_base_vectors[2]).show()
# px.imshow(np.reshape(new_vectors[2], map_a.shape)).show()
px.line(x = map_b.raman_shift, y = new_base_vectors[3]).show()
# px.imshow(np.reshape(new_vectors[3], map_a.shape)).show()
px.line(x = map_b.raman_shift, y = new_base_vectors[4]).show()
# px.imshow(np.reshape(new_vectors[4], map_a.shape)).show()
px.line(x = map_b.raman_shift, y = new_base_vectors[5]).show()
# px.imshow(np.reshape(new_vectors[5], map_b.shape)).show()
px.line(x = map_b.raman_shift, y = new_base_vectors[6]).show()
# px.imshow(np.reshape(new_vectors[6], map_b.shape)).show()


In [ ]:
map_b.plotSpectrum(map_b.posToId(11, 10))